In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.9126


In [6]:
import numpy as np
import csv

class reader(object):
  def __init__(self, data_file = "./data/wdbc.data"):
    self.value = []
    with open(data_file, "rb") as f:
      csv_reader = csv.reader(f, delimiter=",")
      for i, row in enumerate(csv_reader):
        self.value.append(row)
    # print self.value[0]
    # print self.value[0][0]
    # print self.value[0][1]

    self.raw_to_vector(self.value)
    self.split(num_validation_examples=50)
    
#     print self.id.shape
    print self.x.shape
    print self.y.shape
    
    self.num_examples = len(self.x_train)
    self.start_index = 0
    self.shuffle_indices = range(self.num_examples)

    self.num_examples_val = len(self.x_val) # = 50 (validation_data)
    self.start_index_val = 0
    self.shuffle_indices_val = range(self.num_examples_val)
    print self.num_examples_val

  def raw_to_vector(self, value):
    self.id = []
    self.x = []
    self.y = []

    for row in self.value:
        x = np.zeros(30)
        for i in range(30):
            x[i] = float(row[i+2])
        if row[1] == "B":
            y = 0
        else:
            y = 1
        self.x.append(x)
        self.y.append(y)
        id = int(row[0])
        self.id.append(id)
        
    self.x, self.y, self.id = np.array(self.x), np.array(self.y), np.array(self.id)

  def split(self, num_validation_examples):
    self.x_train = self.x[ num_validation_examples: ]
    self.x_val = self.x[ : num_validation_examples ]

    self.y_train = self.y[ num_validation_examples: ]
    self.y_val = self.y[ : num_validation_examples ]

    self.id_train = self.id[ num_validation_examples: ]
    self.id_val = self.id[ :num_validation_examples ]

  def next_batch(self, batch_size, split="train"):

    if split == "train":
      if self.start_index == 0:
        np.random.shuffle(self.shuffle_indices) # shuffle indices

      end_index = min([self.num_examples, self.start_index + batch_size])
      batch_indices = [ self.shuffle_indices[idx] for idx in range(self.start_index, end_index) ]

      batch_x = self.x_train[ batch_indices ]
      batch_y = self.y_train[ batch_indices ]
      batch_id = self.id_train[ batch_indices ] 

      if end_index == self.num_examples:
        self.start_index = 0
      else: self.start_index = end_index

      return batch_x, batch_y, batch_id 

    elif split == "val":
      if self.start_index_val == 0:
        np.random.shuffle(self.shuffle_indices_val) # shuffle indices

      end_index = min([self.num_examples_val, self.start_index_val + batch_size])
      batch_indices = [ self.shuffle_indices_val[idx] for idx in range(self.start_index_val, end_index) ]

      batch_x = self.x_val[ batch_indices ]
      batch_y = self.y_val[ batch_indices ]
      batch_id = self.id_val[ batch_indices ] 

      if end_index == self.num_examples_val:
        self.start_index_val = 0
      else: self.start_index_val = end_index

      return batch_x, batch_y, batch_id  

In [7]:
a = reader()


(569, 30)
(569,)
50


In [9]:
#################
### Logistic Regression with  Tensorflow
#################
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import datasets, model_selection, metrics
import math


### define hyperparameters
n_classes = 2
n_features = 30

learning_rate = 0.05
max_iter = 10000

### define graph
x = tf.placeholder(tf.float32, shape=[None, n_features], name="X")
y_label = tf.placeholder(tf.int32, shape=[None], name="Y_label")
# one-hot encoding
y = tf.one_hot(indices=y_label, depth=n_classes)

# hidden 1
W = tf.Variable(
      tf.truncated_normal([n_features, n_classes],
                          stddev=1.0),
      name='weights')
B = tf.Variable(
      tf.zeros([n_classes]),
      name='biases')

logits = tf.matmul(x, W) + B
scores = tf.nn.softmax( logits )


pred = tf.cast( tf.argmax(logits, 1), tf.int32 )
accuracy = tf.reduce_mean( tf.cast( tf.equal(pred, y_label), tf.float32 ))

### define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)





### define input data
# Load the iris dataset
# iris = datasets.load_iris()
data1 = reader()

# iris has two attributes: data, target
# print(iris.data.shape)
# print(iris.target.shape)

# # in order to plot data, we select first two features
# iris.data = iris.data[:,:2]
# # for binary classification, set label 2 as true class while others are false class
# iris.target[ iris.target != 2 ] = -1
# iris.target[ iris.target == 2 ] = 1
# iris.target[ iris.target == -1 ] = 0

# Split the data into training/testing sets
x_train, x_test, y_train, y_test = model_selection.train_test_split(
  data1.x, data1.y, test_size=0.2, random_state=0)
print x_train.shape
print x_test.shape
print y_train.shape
print y_test.shape


### Starting sessions
with tf.Session() as sess:
  ## initialize variables
  init = tf.global_variables_initializer()
  sess.run(init)

  ## training
  for i in range(max_iter):
    _, accuracy_val, pred_val, loss_val, W_val, B_val = sess.run(
      [train_step, accuracy, pred, loss, W, B], 
      feed_dict={x: x_train, y_label: y_train})

    if i % 100 == 0:
      print('=========== Epoch: %d ===========' % i)
      print('Loss', loss_val)
      print('Accuracy', accuracy_val)
      print('Y_prediction ', pred_val[:10])
      print('True', y_train[:10])
      print('W', W_val)
      print('B', B_val)


    if math.isnan(loss_val):
      print('LOSS is NAN!')
      break

(569, 30)
(569,)
50
(455, 30)
(114, 30)
(455,)
(114,)
=========== Epoch: 0 ===========
('Loss', 731.99146)
('Accuracy', 0.63736266)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
('W', array([[  0.8311491 ,   0.08815062],
       [ -1.99020994,   1.03790879],
       [ -2.1532166 ,   1.53281832],
       [-18.07001114,  16.63008308],
       [ -0.93982774,   0.67846888],
       [  1.60155332,   0.22875381],
       [ -0.08854013,   0.70461982],
       [  0.14447318,  -0.19227797],
       [ -0.45829213,  -1.36855292],
       [  1.17950261,   0.02937623],
       [ -0.4842484 ,   0.71492213],
       [  0.06318067,  -0.04288561],
       [ -0.88198161,   0.5257839 ],
       [ -1.10302567,   1.87715387],
       [  0.04518936,  -1.10609555],
       [ -1.51315999,  -0.4168286 ],
       [  0.42282736,   1.83286405],
       [  0.32445675,  -0.82046914],
       [  0.7760154 ,  -0.69691432],
       [ -1.13408804,  -0.68261427],
    

=========== Epoch: 700 ===========
('Loss', 367.05618)
('Accuracy', 0.91208792)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
('W', array([[  1.43547850e+01,  -1.34354887e+01],
       [  1.93409138e+01,  -2.02932014e+01],
       [  7.81428833e+01,  -7.87632751e+01],
       [  5.08056793e+01,  -5.22455978e+01],
       [ -8.11250567e-01,   5.49891710e-01],
       [  1.58097553e+00,   2.49327749e-01],
       [ -2.67736942e-01,   8.83820057e-01],
       [  6.43488839e-02,  -1.12153776e-01],
       [ -2.16890007e-01,  -1.60996068e+00],
       [  1.27950740e+00,  -7.06291199e-02],
       [ -4.52067077e-01,   6.82738483e-01],
       [  1.51003134e+00,  -1.48973620e+00],
       [ -1.13319159e+00,   7.76997924e-01],
       [ -3.75700493e+01,   3.83441734e+01],
       [  5.46189696e-02,  -1.11552835e+00],
       [ -1.51826048e+00,  -4.11736608e-01],
       [  4.12871897e-01,   1.84281695e+00],
       [  3.26304048e-01,  -8.2

=========== Epoch: 1500 ===========
('Loss', 274.24063)
('Accuracy', 0.92087913)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
('W', array([[  1.60869789e+01,  -1.51677332e+01],
       [  1.54020662e+01,  -1.63543701e+01],
       [  8.45989227e+01,  -8.52193146e+01],
       [  3.08158283e+01,  -3.22557564e+01],
       [ -8.17260921e-01,   5.55902064e-01],
       [  1.46581554e+00,   3.64486545e-01],
       [ -4.48101282e-01,   1.06417775e+00],
       [ -6.01452542e-03,  -4.17902693e-02],
       [ -2.36573055e-01,  -1.59027767e+00],
       [  1.28165722e+00,  -7.27798641e-02],
       [ -4.91462678e-01,   7.22128451e-01],
       [  1.04419923e+00,  -1.02390409e+00],
       [ -2.01555514e+00,   1.65936363e+00],
       [ -4.82529411e+01,   4.90270653e+01],
       [  5.24134487e-02,  -1.11331356e+00],
       [ -1.55387712e+00,  -3.76130104e-01],
       [  3.67465109e-01,   1.88822222e+00],
       [  3.15923989e-01,  -8.

=========== Epoch: 2300 ===========
('Loss', 256.98071)
('Accuracy', 0.90989012)
('Y_prediction ', array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
('W', array([[  1.73663883e+01,  -1.64471493e+01],
       [  1.02276211e+01,  -1.11799355e+01],
       [  8.83089371e+01,  -8.89293289e+01],
       [  2.35851994e+01,  -2.50251541e+01],
       [ -8.30646992e-01,   5.69288135e-01],
       [  1.34417927e+00,   4.86114055e-01],
       [ -6.27548337e-01,   1.24362302e+00],
       [ -7.47086853e-02,   2.69039944e-02],
       [ -2.72583991e-01,  -1.55426264e+00],
       [  1.27856767e+00,  -6.96855411e-02],
       [ -5.20120203e-01,   7.50785589e-01],
       [  5.85947514e-01,  -5.65653324e-01],
       [ -2.87327647e+00,   2.51708627e+00],
       [ -5.63977509e+01,   5.71718750e+01],
       [  5.01510054e-02,  -1.11104393e+00],
       [ -1.58735800e+00,  -3.42646718e-01],
       [  3.23650897e-01,   1.93203962e+00],
       [  3.05838138e-01,  -8.

=========== Epoch: 3000 ===========
('Loss', 236.68637)
('Accuracy', 0.92747253)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
('W', array([[  1.85962334e+01,  -1.76769943e+01],
       [  6.09906054e+00,  -7.05136585e+00],
       [  9.22682800e+01,  -9.28886719e+01],
       [  2.10600166e+01,  -2.25000038e+01],
       [ -8.42136204e-01,   5.80777347e-01],
       [  1.23792267e+00,   5.92366219e-01],
       [ -7.84401119e-01,   1.40047669e+00],
       [ -1.35099456e-01,   8.72952491e-02],
       [ -3.02004844e-01,  -1.52484024e+00],
       [  1.27618790e+00,  -6.73078001e-02],
       [ -5.30253828e-01,   7.60919273e-01],
       [  2.60032445e-01,  -2.39737794e-01],
       [ -3.51647162e+00,   3.16028142e+00],
       [ -6.21498756e+01,   6.29239998e+01],
       [  4.84495647e-02,  -1.10933936e+00],
       [ -1.61534035e+00,  -3.14665347e-01],
       [  2.87037194e-01,   1.96864307e+00],
       [  2.97644198e-01,  -7.

...

=========== Epoch: 6200 ===========
('Loss', 253.26349)
('Accuracy', 0.91428572)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
('W', array([[  2.48381634e+01,  -2.39189243e+01],
       [ -7.86470747e+00,   6.91239071e+00],
       [  1.13580475e+02,  -1.14200867e+02],
       [  2.66349144e+01,  -2.80749226e+01],
       [ -8.98747087e-01,   6.37388229e-01],
       [  7.31920123e-01,   1.09836876e+00],
       [ -1.52269399e+00,   2.13877273e+00],
       [ -4.26700801e-01,   3.78897786e-01],
       [ -4.34285641e-01,  -1.39255345e+00],
       [  1.26550996e+00,  -5.66280223e-02],
       [ -4.39553648e-01,   6.70214891e-01],
       [ -5.75344026e-01,   5.95638096e-01],
       [ -5.46535921e+00,   5.10911322e+00],
       [ -7.50296249e+01,   7.58037643e+01],
       [  4.16259281e-02,  -1.10247314e+00],
       [ -1.74120271e+00,  -1.88812882e-01],
       [  1.26081154e-01,   2.12960172e+00],
       [  2.62194932e-01,  -7.

=========== Epoch: 7000 ===========
('Loss', 256.08344)
('Accuracy', 0.92087913)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
('W', array([[  2.64899902e+01,  -2.55707512e+01],
       [ -1.02915573e+01,   9.33924484e+00],
       [  1.19368568e+02,  -1.19988960e+02],
       [  3.02345104e+01,  -3.16745129e+01],
       [ -9.13659513e-01,   6.52300656e-01],
       [  6.02157533e-01,   1.22813201e+00],
       [ -1.71132302e+00,   2.32741022e+00],
       [ -5.02358496e-01,   4.54555482e-01],
       [ -4.68221545e-01,  -1.35861623e+00],
       [  1.26291466e+00,  -5.40309027e-02],
       [ -4.03777093e-01,   6.34437621e-01],
       [ -6.49739623e-01,   6.70033693e-01],
       [ -5.85578871e+00,   5.49954271e+00],
       [ -7.68611221e+01,   7.76352615e+01],
       [  3.99109013e-02,  -1.10074365e+00],
       [ -1.77222478e+00,  -1.57794923e-01],
       [  8.63337517e-02,   2.16934466e+00],
       [  2.53262550e-01,  -7.

=========== Epoch: 7800 ===========
('Loss', 385.89877)
('Accuracy', 0.91428572)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
('W', array([[  2.61929913e+01,  -2.52737522e+01],
       [ -1.58717365e+01,   1.49194241e+01],
       [  1.13112350e+02,  -1.13732742e+02],
       [  1.86090393e+01,  -2.00490417e+01],
       [ -9.50107574e-01,   6.88748717e-01],
       [  4.63007450e-01,   1.36728394e+00],
       [ -1.89217842e+00,   2.50827551e+00],
       [ -5.73552251e-01,   5.25749922e-01],
       [ -5.42518318e-01,  -1.28431773e+00],
       [  1.24510765e+00,  -3.62263583e-02],
       [ -3.62857431e-01,   5.93519866e-01],
       [ -9.19833899e-01,   9.40127969e-01],
       [ -6.21366978e+00,   5.85742378e+00],
       [ -7.33723831e+01,   7.41465225e+01],
       [  3.63194086e-02,  -1.09714472e+00],
       [ -1.80602300e+00,  -1.24002650e-01],
       [  4.27361839e-02,   2.21295285e+00],
       [  2.42739156e-01,  -7.

=========== Epoch: 8600 ===========
('Loss', 582.59503)
('Accuracy', 0.8967033)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
('W', array([[  2.78659458e+01,  -2.69467068e+01],
       [ -1.76086979e+01,   1.66563759e+01],
       [  1.19227386e+02,  -1.19847771e+02],
       [  1.77032604e+01,  -1.91432476e+01],
       [ -9.62130010e-01,   7.00771153e-01],
       [  3.41254234e-01,   1.48903847e+00],
       [ -2.07306838e+00,   2.68917918e+00],
       [ -6.45281672e-01,   5.97479343e-01],
       [ -5.71982205e-01,  -1.25485349e+00],
       [  1.24467051e+00,  -3.57885435e-02],
       [ -3.12511772e-01,   5.43175638e-01],
       [ -9.33326840e-01,   9.53620911e-01],
       [ -6.49957705e+00,   6.14333105e+00],
       [ -7.42427902e+01,   7.50169296e+01],
       [  3.46844569e-02,  -1.09550381e+00],
       [ -1.83528221e+00,  -9.47448984e-02],
       [  3.71704460e-03,   2.25197840e+00],
       [  2.33949289e-01,  -7.2

=========== Epoch: 9400 ===========
('Loss', 251.08752)
('Accuracy', 0.91208792)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
('W', array([[  3.01310406e+01,  -2.92118015e+01],
       [ -1.79240284e+01,   1.69717731e+01],
       [  1.28935074e+02,  -1.29555435e+02],
       [  2.54463406e+01,  -2.68863010e+01],
       [ -9.69811320e-01,   7.08452463e-01],
       [  2.18095005e-01,   1.61219215e+00],
       [ -2.25995803e+00,   2.87606883e+00],
       [ -7.20679045e-01,   6.72876716e-01],
       [ -5.91896892e-01,  -1.23494077e+00],
       [  1.24774623e+00,  -3.88665237e-02],
       [ -2.53324449e-01,   4.83991861e-01],
       [ -8.34340930e-01,   8.54638517e-01],
       [ -6.71099472e+00,   6.35474873e+00],
       [ -7.54139557e+01,   7.61880951e+01],
       [  3.35749425e-02,  -1.09439230e+00],
       [ -1.86412752e+00,  -6.59063905e-02],
       [ -3.38220820e-02,   2.28952169e+00],
       [  2.25800529e-01,  -7.